# This project will be final capstone of IBM Data Science

## first we should set libraries

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


## Next we should add FourSquare

In [5]:
CLIENT_ID = "DOOH2RDI0SYNUBM3Z3Q2YA2KWLZZ3RS3SHJUWOVLHHMGARV1"
CLIENT_SECRET = "MQYN1QOTZ1RVCRQXZ24KAHFL1URDVZG1RVKC0ZNXD0AYACGK"

VERSION = "20210305"
LIMIT = 20

print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: DOOH2RDI0SYNUBM3Z3Q2YA2KWLZZ3RS3SHJUWOVLHHMGARV1
CLIENT_SECRET:MQYN1QOTZ1RVCRQXZ24KAHFL1URDVZG1RVKC0ZNXD0AYACGK


In [22]:
# checking coordinates of Colombia University
address = "116th and Broadway, New York, NY 10027" 

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The latitude and longitude coordinates are:")
print(latitude, longitude)

The latitude and longitude coordinates are:
40.8175378 -73.9568702


In [23]:
# lets find university building around center
search_query = "College Academic Building"
radius = 5000 #Radius of search in meters

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

In [24]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a pandas dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4bfda08d83bbd13a4a0a63c5,Administration Building - City College,"[{'id': '4bf58dd8d48988d197941735', 'name': 'C...",v-1614967562,False,160 Convent Ave,140th St,40.820134,-73.948969,"[{'label': 'display', 'lat': 40.82013442621451...",725,10031,US,New York,NY,United States,"[160 Convent Ave (140th St), New York, NY 1003..."
1,4bbcf07f8a4fb713c8603c9d,Compton-Goethals Building - The City College o...,"[{'id': '4bf58dd8d48988d199941735', 'name': 'C...",v-1614967562,False,NaN,NaN,40.821921,-73.949750,"[{'label': 'display', 'lat': 40.82192082901643...",773,10031,US,New York,NY,United States,"[New York, NY 10031, United States]"
2,4ebc75e693ad3578f03e531f,Hostos Community College C - Building (East A...,"[{'id': '4bf58dd8d48988d19f941735', 'name': 'C...",v-1614967562,False,450 Grandconcourse,NaN,40.817485,-73.927155,"[{'label': 'display', 'lat': 40.81748476326776...",2503,10451,US,Bronx,NY,United States,"[450 Grandconcourse, Bronx, NY 10451, United S..."
3,5808ea4b38fad30e301f4dda,Hostos Community College (CUNY) D Building (Sa...,"[{'id': '4bf58dd8d48988d197941735', 'name': 'C...",v-1614967562,False,120 East 149th Street,Walton Avenue,40.818798,-73.928455,"[{'label': 'display', 'lat': 40.818798, 'lng':...",2397,10459,US,Bronx,NY,United States,"[120 East 149th Street (Walton Avenue), Bronx,..."


In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Administration Building - City College,College Administrative Building,160 Convent Ave,140th St,40.820134,-73.948969,"[{'label': 'display', 'lat': 40.82013442621451...",725,10031,US,New York,NY,United States,"[160 Convent Ave (140th St), New York, NY 1003...",4bfda08d83bbd13a4a0a63c5
1,Compton-Goethals Building - The City College o...,College Arts Building,NaN,NaN,40.821921,-73.949750,"[{'label': 'display', 'lat': 40.82192082901643...",773,10031,US,New York,NY,United States,"[New York, NY 10031, United States]",4bbcf07f8a4fb713c8603c9d
2,Hostos Community College C - Building (East A...,College Technology Building,450 Grandconcourse,NaN,40.817485,-73.927155,"[{'label': 'display', 'lat': 40.81748476326776...",2503,10451,US,Bronx,NY,United States,"[450 Grandconcourse, Bronx, NY 10451, United S...",4ebc75e693ad3578f03e531f
3,Hostos Community College (CUNY) D Building (Sa...,College Administrative Building,120 East 149th Street,Walton Avenue,40.818798,-73.928455,"[{'label': 'display', 'lat': 40.818798, 'lng':...",2397,10459,US,Bronx,NY,United States,"[120 East 149th Street (Walton Avenue), Bronx,...",5808ea4b38fad30e301f4dda


## Visualizing Data

In [29]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Grand Central Terminal

# add a red circle marker to represent Grand Central Terminal
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Grand Central Terminal',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the pizza joints as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [28]:
# Conclusion: we searched location of colombia university in the map with using Foursquare api besides 
# we can see another academic building location aroun Colombia university